<span style="display:block;text-align:center;margin-right:105px"><img src="../../media/logos/logo-vertical.png" width="200"/></span>

# Section 7: Capstone Project

---

# Table of Contents

* [System Requirements (Part 1)](#System-Requirements)
  * [Model Introduction](#Model-Introduction)
  * [Requirements Analysis](#Requirements-Analysis)
  * [Visual System Mapping: Causal Loop Diagram](#Visual-System-Mapping:-Causal-Loop-Diagram)
  * [Visual System Mapping: Stock & Flow Diagram](#Visual-System-Mapping:-Stock-&-Flow-Diagram)
  * [Mathematical Specification](#Mathematical-Specification)
* [System Design (Part 2)](#System-Design)
  * [Differential Specification](#Differential-Specification)
  * [cadCAD Standard Notebook Layout](#cadCAD-Standard-Notebook-Layout)
    0. [Dependencies](#0.-Dependencies)
    1. [State Variables](#1.-State-Variables)
    2. [System Parameters](#2.-System-Parameters)
    3. [Policy Functions](#3.-Policy-Functions)
    4. [State Update Functions](#4.-State-Update-Functions)
    5. [Partial State Update Blocks](#5.-Partial-State-Update-Blocks)
    6. [Configuration](#6.-Configuration)
    7. [Execution](#7.-Execution)
    8. [Simulation Output Preparation](#8.-Simulation-Output-Preparation)
* [System Validation (Part 3)](#System-Validation)
  * [What-if Matrix](#What-if-Matrix)
  * [System Analysis](#System-Analysis)

---

# System Requirements

<center><img src="images/edp-phase-1.png" alt="Engineering Design Process, phase 1 - System requirements" width="60%"/>

## Model Introduction

<center>
<img src="images/globe.png" alt="Earth globe" width="200px"/>

Project Anthropocene is a model that enables the insightful analysis of the impact of carbon dioxide (CO2) on the Earth's temperature.

## Requirements Analysis

[Link to System Analysis](#System-Analysis)

### Questions
       
**Planned analysis:** How does the Earth's temperature evolve over the next 100 years under various assumptions regarding CO2 emissions?

1. How will the __Earth's average temperature__ and the __rate of annual temperature change__ develop over the next 100 years, if we keep CO2 emissions __unchanged__ at today’s annual emission levels vs. a __doubling__ of today’s emission levels.
2. How will the __Earth's average temperature__ and the __rate of annual temperature change__ develop over the next 100 years if we are able to reduce annual CO2 emissions to __zero__ after a given number of years?

## Visual System Mapping: Causal Loop Diagram

The overall __relationships__ in the model are the following:
* The __Earth's temperature is determined by what's called radiation balance__, i.e. how much radiation comes in via the Sun, minus how much is dissipating into space. If this balance is positive, heat accumulates, and the Earth warms up; if it is negative, the Earth cools down.
* The __radiation balance__ is driven by the Sun's radiation, which tends to make the Earth hotter, and the Earth's radiation, which makes heat dissipate and the planet colder.
* The __radiation balance is influenced by the well-known greenhouse effect__, i.e. the stronger the greenhouse effect, the more radiation from Earth gets trapped in the atmosphere unable to dissipate into space and the higher the radiation balance. Quick primer on the greenhouse effect: https://en.wikipedia.org/wiki/Greenhouse_effect
* __CO2__ contributes strongly to the greenhouse effect.

<center>
<img src="images/s7-climate-cld-diagram.png" alt="Model for Project Anthropocene" width="60%"/>

## Visual System Mapping: Stock & Flow Diagram

<center>
<img src="images/s7-anthropocene-stock-and-flow.png" alt="Stock & Flow diagram for Project Anthropocene" width="60%"/>

## Mathematical Specification


The Anthropocene system is an IVP (initial value problem) which is described by the following equations:

\begin{align}
\tag{1}
dCO_2(t) = \begin{cases}
    \mathcal{N}(\mu,  \sigma) & \forall t \in [0, t_w] \\
     \mathcal{N}(0,  \sigma) & \forall t \in [t_w, \infty]
            \end{cases}
\end{align}

\begin{align}
\tag{2}
\alpha(t) = 1 - e^{-\beta * CO_2(t)}
\end{align}

\begin{align}
\tag{3}
Y(t) = \alpha(t) Z(t)
\end{align}

\begin{align}
\tag{4}
Z(t) = K T(t)^4
\end{align}

\begin{align}
\tag{5}
dT(t) = \gamma(a + Y(t) - Z(t))
\end{align}


Where $\mathcal{N}$ represents the normal distribution with mean $\mu$ and standard deviation $\sigma$. For each timestep $t$ we have the $CO_2(t)$, $\alpha(t)$, $Y(t)$, $T(t)$ and $Z(t)$ being the atmospheric $CO_2$ concentration, the atmosphere reflectance, the reflected radiation, the surface temperature and the outgoing radiation respectively. Also, we have the $\beta$, $\gamma$, $a$, $K$, $t_w$ constants as being a $CO_2$ to reflectance conversion factor, a radiation to temperature conversion factor, the Sun's yearly radiance, a constant for the blackbody effect and the year where emissions are stopped.

The system is tightly coupled and is both non-linear and stochastic, which can make mathematical treatment difficult, and as such, the characterization of it will be made easier through an experimental approach and computational simulations, which is exactly what **cadCAD** enables us to do.

# System Design

<center><img src="images/edp-phase-2.png" alt="Engineering Design Process, phase 1 - System design" width="60%"/>

## Differential Specification

<center><img src="images/s7-differential-spec-anthropocene.png" alt="Model for Project Anthropocene" width="60%"/>

## cadCAD Standard Notebook Layout

### 0. Dependencies

In [1]:
import pandas as pd
import numpy as np
from random import normalvariate
import plotly.express as px

from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD.engine import ExecutionContext, Executor

### 1. State Variables

The states we are interested in, their state variables and their initial values are:

* The __atmosphere's CO2 concentration__ in parts per million (ppm): `co2`, initial value 400
* The __earth's surface temperature__ in Kelvin (K): `temperature`, initial value 290
    
<!--**Create a dictionary and define the above state variables and their initial values:**-->

In [2]:
initial_state = {
    'co2': 400,
    'temperature': 290
}

### 2. System Parameters

**The system parameters we need to define are:**

* The sun radiation: `sun_radiation` with value `1361`
* A constant representing the relationship between temperature and radiation: `temperature_constant` with value `1e-4`
* A constant representing CO2 impact on the radiation balance via the greenhouse effect: `co2_reflectance_factor` with value `1e-3`
* A unit conversion constant that relates how much gigatons of CO2 we need to have an additional part per million unit in the atmosphere's concentration: `co2_gigatons_to_ppm` with value `1.2e-1`
* The standard deviation for the stochastic process generating the yearly CO2 concentration: `co2_stdev` with value `40` ppm
* A constant representing how much heat dissipitates into space: `heat_dissipation_constant` with value `2075`

**There are two parameters which we want to sweep, which are:**

* A parameter which represents the annual CO2 emissions in units of billion tons, which is the `co2_annual_emissions`. Let's sweep three values for it: `0`, `40` and `80`. The first value simulates a scenario where we stop all emissions at once, while using `40` means no additional emissions beyond what we already emmit every year, and `80` means that we are going to double our emissions.
* The `year_of_the_wakening`, which is the number of years that must pass before we set the `co2_annual_emissions` to zero. Let's sweep four values for it: `0`, `10`, `50` and `100`.

<!--**Create a dictionary and define the above parameters and their initial values:**-->

In [3]:
system_params = {
    'sun_radiation': [1361], # solar constant, the average amount of solar radiation received at the outer atmosphere of the Earth on a unit area perpendicular to the Sun's rays in W/m²
    'temperature_constant': [1e-4],
    'co2_reflectance_factor': [1e-3],
    'co2_gigatons_to_ppm': [1.2e-1],
    'co2_stdev': [40],
    'heat_dissipation_constant': [2075],
    'co2_annual_emissions': [40, 80, 40, 80, 40, 80, 40, 80],
    'year_of_the_wakening': [0, 0, 10, 10, 50, 50, 100, 100]
}

In [4]:
assert 1e10 == 1*10**10

### 3. Policy Functions

In [5]:
def p_co2_emissions(params, subbstep, state_history, previous_state):
    # Parameters & variables
    mean = params['co2_annual_emissions']
    std = params['co2_stdev']
    conversion_factor = params['co2_gigatons_to_ppm']
    t_w = params['year_of_the_wakening']
    t = previous_state['timestep']
    
    # Logic
    if t > t_w:
        mean = 0
    else:
        mean = mean
    value = normalvariate(mean, std) * conversion_factor

    # Output
    return {'add_co2': value}

In [6]:
def p_sun_radiation(params, substep, state_history, previous_state):
    # Parameters & variables
    g = params['temperature_constant']
    a = params['sun_radiation']
    
    # Logic
    temp_change = g * a
    
    # Output
    return {'add_temperature': temp_change}

In [7]:
def p_earth_cooling(params, substep, state_history, previous_state):
    # Parameters & variables
    g = params['temperature_constant']
    K = params['heat_dissipation_constant']
    T = previous_state['temperature']
    
    # Logic
    temp_change = -(g * K * (T / 300) ** 4)
    
    # Output
    return {'add_temperature': temp_change}

In [8]:
def p_greenhouse_effect(params, substep, state_history, previous_state):
    # Parameters & variables
    g = params['temperature_constant']
    K = params['heat_dissipation_constant']
    beta = params['co2_reflectance_factor']
    T = previous_state['temperature']
    CO2 = previous_state['co2']
    
    # Logic
    alpha = (1 - np.exp(-beta * CO2))
    temp_change = g * alpha * K * (T / 300) ** 4
    
    # Output
    return {'add_temperature': temp_change}

### 4. State Update Functions

In [9]:
def s_co2(params, substep, state_history, previous_state, policy_input):
    # Parameters & variables
    current_co2 = previous_state['co2']
    co2_change = policy_input['add_co2']
    
    # Logic
    new_co2 = max(current_co2 + co2_change, 0)
    
    # Output
    return ('co2', new_co2)

In [10]:
def s_temperature(params, substep, state_history, previous_state, policy_input):
    # Parameters & variables
    current_temp = previous_state['temperature']
    temp_change = policy_input['add_temperature']
    
    # Logic
    new_temp = max(current_temp + temp_change, 0)
    
    # Output
    return ('temperature', new_temp)

### 5. Partial State Update Blocks

In [11]:
partial_state_update_blocks = [
    {
        'label': 'Temperature dynamics', # Useful metadata to describe our partial state update blocks
        'policies': {
            'sun_radiation': p_sun_radiation,
            'earth_cooling': p_earth_cooling,
            'greenhouse_effect': p_greenhouse_effect
        },
        'variables': {
            'temperature': s_temperature
            
        }
    },
    {
        'label': 'CO2 dynamics', # Useful metadata to describe our partial state update blocks
        'policies': {
            'co2_emissions': p_co2_emissions
        },
        'variables': {
            'co2': s_co2
        }
        
    }
]


### 6. Configuration

In [12]:
MONTE_CARLO_RUNS = 50
SIMULATION_TIMESTEPS = 100

sim_config = config_sim(
    {
        'N': MONTE_CARLO_RUNS,
        'T': range(SIMULATION_TIMESTEPS),
        'M': system_params, # parameters to sweep over
    }
)

from cadCAD import configs
del configs[:] # Clear any prior configs

experiment = Experiment()
experiment.append_configs(
    sim_configs=sim_config,
    initial_state=initial_state,
    partial_state_update_blocks=partial_state_update_blocks
)

### 7. Execution

In [16]:
exec_context = ExecutionContext()
run = Executor(exec_context=exec_context, configs=configs)

(system_events, tensor_field, sessions) = run.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 8
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (100, 8, 50, 2)
Execution Method: local_simulations
SimIDs   : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4

### 8. Simulation Output Preparation

In [17]:
# Get system events and attribute index
df = (pd.DataFrame(system_events)
        .assign(years=lambda df: df.timestep)
        .assign(temperature_celsius=lambda df: df.temperature - 273)
        .query('timestep > 0')
     )

# Clean substeps
# Aside from each point in time (timesteps), cadCAD generates points for each intermediate execution through the PSUBs.
# These points are known as substeps. If 3 blocks, then 3 substeps per timestep.

first_ind = (df.substep == 0) & (df.timestep == 0)
last_ind = df.substep == max(df.substep)
inds_to_drop = (first_ind | last_ind)
df = df.loc[inds_to_drop].drop(columns=['substep'])

# Attribute parameters to each row
df = df.assign(**configs[0].sim_config['M'])
for i, (_, n_df) in enumerate(df.groupby(['simulation', 'subset', 'run'])):
    df.loc[n_df.index] = n_df.assign(**configs[i].sim_config['M'])

df

,co2,temperature,simulation,subset,run,timestep,years,temperature_celsius,sun_radiation,temperature_constant,co2_reflectance_factor,co2_gigatons_to_ppm,co2_stdev,heat_dissipation_constant,co2_annual_emissions,year_of_the_wakening
2,405.967082,290.014647,0,0,1,1,1,17.014647,1361,0.0001,0.001,0.12,40,2075,40,0
4,402.350132,290.029993,0,0,1,2,2,17.029993,1361,0.0001,0.001,0.12,40,2075,40,0
6,405.345422,290.044875,0,0,1,3,3,17.044875,1361,0.0001,0.001,0.12,40,2075,40,0
8,403.322168,290.060095,0,0,1,4,4,17.060095,1361,0.0001,0.001,0.12,40,2075,40,0
10,406.881662,290.075045,0,0,1,5,5,17.075045,1361,0.0001,0.001,0.12,40,2075,40,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80391,1326.222544,295.344811,7,7,50,96,96,22.344811,1361,0.0001,0.001,0.12,40,2075,80,100
80393,1333.351821,295.429165,7,7,50,97,97,22.429165,1361,0.0001,0.001,0.12,40,2075,80,100
80395,1331.387948,295.513828,7,7,50,98,98,22.513828,1361,0.0001,0.001,0.12,40,2075,80,100
80397,1341.952039,295.598330,7,7,50,99,99,22.598330,1361,0.0001,0.001,0.12,40,2075,80,100


# System Validation

<center><img src="images/edp-phase-3.png" alt="Engineering Design Process, phase 1 - validation" width="60%"/>

[Link to Requirements Analysis](#Requirements-Analysis)

## What-if Matrix

What-if-question | Type of experiment | Variables / parameters | Values / Ranges to be tested
- | - | - | -
How will the __Earth's average temperature__ develop over the next 100 years, if we keep CO2 emissions __unchanged__ at today’s annual emission levels vs. a __doubling__ of today’s emission levels? | Parameter Sweep + Monte Carlo runs | co2_annual_emissions | 40 and 80 Gigatons
How will the __rate of annual temperature change__ develop over the next 100 years if we keep CO2 emissions __unchanged__ at today’s annual emission levels vs. a __doubling__ of today’s emission levels? | Parameter Sweep + Monte Carlo runs | co2_annual_emissions | 40 and 80 Gigatons
How will the __rate of annual temperature change__ develop over the next 100 years if we are able to reduce annual CO2 emissions to __zero__ after a given number of years? | Parameter Sweep + Monte Carlo runs | year_of_the_wakening | 0, 10, 50 and 100 years
How will the __Earth's average temperature__ develop over the next 100 years if we are able to reduce annual CO2 emissions to __zero__ after a given number of years? | Parameter Sweep + Monte Carlo runs | year_of_the_wakening | 0, 10, 50 and 100 years

## System Analysis

### Analysis 1: How will the Earth's average temperature develop over the next 100 years, if we keep CO2 emissions unchanged at today’s annual emission levels vs. a doubling of today’s emission levels?

In [18]:
fig_df = df.query('year_of_the_wakening == 100')

fig = px.scatter(
    fig_df,
    x=fig_df.years,
    y=fig_df.temperature_celsius,
    color=fig_df.co2_annual_emissions.astype(str),
    opacity=0.1,
    trendline="lowess",
    labels={'color': 'Yearly CO2 emissions (Gt)'}
)

fig.show()

AttributeError: module 'numpy' has no attribute 'MachAr'

In [20]:
fig_df = df.query('year_of_the_wakening == 100')

fig = px.box(
    fig_df,
    x=fig_df.years,
    y=fig_df.temperature_celsius,
    color=fig_df.co2_annual_emissions.astype(str),
    points=False,
    labels={'color': 'Yearly CO2 emissions (Gt)'}
)

fig.show()

### Analysis 2: How will the rate of annual temperature change develop over the next 100 years if we keep CO2 emissions unchanged at today’s annual emission levels vs. a doubling of today’s emission levels?

In [21]:
fig_df = (df.query('year_of_the_wakening == 100')
            .assign(annual_temperature_increase=lambda df: df.temperature.diff())
            .query('years > 1'))

fig = px.scatter(
    fig_df,
    x=fig_df.years,
    y=fig_df.annual_temperature_increase,
    opacity=0.1,
    trendline="lowess",
    color=fig_df.co2_annual_emissions.astype(str),
    labels={'color': 'Yearly CO2 emissions (Gt)'}
)

fig.show()

AttributeError: module 'numpy' has no attribute 'MachAr'

### Analysis 3: How will the rate of annual temperature change develop over the next 100 years if we are able to reduce annual CO2 emissions to zero after a given number of years?

In [22]:
fig_df = (df.query('co2_annual_emissions == 40')
            .assign(annual_temperature_increase=lambda df: df.temperature.diff())
            .query('years > 1'))

fig = px.scatter(
    fig_df,
    x=fig_df.years,
    y=fig_df.annual_temperature_increase,
    opacity=0.1,
    trendline="lowess",
    color=fig_df.year_of_the_wakening.astype(str),
    labels={'color': 'Year of the wakening (years)'}
)

fig.show()

AttributeError: module 'numpy' has no attribute 'MachAr'

### Analysis 4: How will the Earth's average temperature develop over the next 100 years if we are able to reduce annual CO2 emissions to zero after a given number of years?

In [23]:
fig_df = (df.query('co2_annual_emissions == 40')
            .assign(temperature_increase=lambda df: df.temperature - df.temperature.iloc[0]))

fig = px.scatter(
    fig_df,
    x=fig_df.years,
    y=fig_df.temperature_increase,
    opacity=0.1,
    trendline="lowess",
    color=fig_df.year_of_the_wakening.astype(str),
    labels={'color': 'Year of the wakening (years)'}
)

fig.show()

AttributeError: module 'numpy' has no attribute 'MachAr'

# Congratulations!

![](./images/climate-model.png)